In [1]:
import torch #Menyediakan fungsi dasar untuk operasi tensor dan komputasi berbasis GPU.
import torch.nn as nn #Menyediakan modul dan kelas untuk membangun jaringan saraf.
import torch.optim as optim #Menyediakan algoritma pengoptimalan untuk melatih model.
from torch.utils.data import DataLoader, random_split #Menyediakan kelas dan fungsi untuk memuat dan memproses data.
import torchvision #Menyediakan kumpulan data, model, dan transformasi gambar yang telah dibuat sebelumnya.
import torchvision.transforms as transforms #Menyediakan transformasi gambar umum untuk augmentasi data.
from torch.optim.lr_scheduler import ReduceLROnPlateau #Menyediakan scheduler laju pembelajaran untuk menyesuaikan laju pembelajaran selama pelatihan.

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
#Transformasi Dataset

In [3]:
batch_size = 32
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#Load CIFAR-10 Dataset

100%|██████████| 170M/170M [00:04<00:00, 35.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#Membagi dataset pelatihan menjadi dataset pelatihan dan validasi

In [5]:
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        padding = kernel_size // 2  # Menentukan padding agar dimensi output tetap
        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=padding)  # Layer konvolusi pertama
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)  # Layer konvolusi kedua

        # Menentukan jenis pooling (max pooling atau average pooling)
        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)  # Average pooling

        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Fully connected layer pertama
        self.fc2 = nn.Linear(128, 10)  # Fully connected layer kedua (output)
        self.relu = nn.ReLU()  # Fungsi aktivasi ReLU
        self.dropout = nn.Dropout(0.5)  # Dropout untuk mencegah overfitting

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # Kombinasi konvolusi, ReLU, dan pooling (layer pertama)
        x = self.pool(self.relu(self.conv2(x)))  # Kombinasi konvolusi, ReLU, dan pooling (layer kedua)
        x = x.view(-1, 64 * 8 * 8)  # Flatten tensor untuk input ke fully connected layer
        x = self.relu(self.fc1(x))  # Fully connected layer pertama dengan ReLU
        x = self.dropout(x)  # Dropout
        x = self.fc2(x)  # Fully connected layer kedua (output)
        return x
#CNN Arsitektur

In [ ]:
# Proses Training dan Evaluasi
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs, early_stopping_patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Menentukan perangkat yang digunakan (GPU atau CPU)
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)  # Inisialisasi model dengan parameter tertentu
    criterion = nn.CrossEntropyLoss()  # Menggunakan fungsi loss CrossEntropy

    # Konfigurasi optimizer
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # Optimizer SGD
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)  # Optimizer RMSProp
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer Adam

    # Scheduler untuk penyesuaian learning rate
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)

    best_loss = float('inf')  # Inisialisasi nilai loss terbaik
    patience_counter = 0  # Penghitung untuk early stopping

    print(f"Evaluating with params: {{'epochs': {epochs}, 'kernel_size': {kernel_size}, 'optimizer_type': '{optimizer_type}', 'pooling_type': '{pooling_type}'}}")

    for epoch in range(epochs):
        # Proses Training
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Mengosongkan gradien sebelumnya
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Menghitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Memperbarui bobot model

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)  # Rata-rata loss selama training

        # Proses Validasi
        model.eval()
        val_loss = 0.0
        with torch.no_grad():  # Menonaktifkan gradient computation
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)  # Forward pass
                loss = criterion(outputs, labels)  # Menghitung loss
                val_loss += loss.item()
        val_loss /= len(val_loader)  # Rata-rata loss validasi

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)  # Memperbarui learning rate berdasarkan validasi loss

        # Mekanisme Early Stopping
        if val_loss < best_loss:
            best_loss = val_loss  # Menyimpan nilai loss terbaik
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:  # Menghentikan training jika tidak ada perbaikan
                print("Early stopping triggered.")
                break

    # Evaluasi akurasi pada dataset uji
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Mendapatkan prediksi
            total += labels.size(0)  # Total sample
            correct += (predicted == labels).sum().item()  # Jumlah prediksi benar

    accuracy = 100 * correct / total  # Menghitung akurasi
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Eksperimen dengan berbagai kombinasi parameter
kernel_sizes = [3, 5, 7]  # Variasi ukuran kernel
pooling_types = ['max', 'avg']  # Variasi jenis pooling
optimizers = ['SGD', 'RMSProp', 'Adam']  # Variasi optimizer
epoch_list = [5, 50, 100, 250, 350]  # Variasi jumlah epoch

results = []

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epoch_list:
                print(f"Running experiment with kernel_size={kernel_size}, pooling_type={pooling_type}, optimizer={optimizer}, epochs={epochs}")
                accuracy = train_and_evaluate(kernel_size, pooling_type, optimizer, epochs, early_stopping_patience=10)  # Melatih dan mengevaluasi model
                results.append((kernel_size, pooling_type, optimizer, epochs, accuracy))  # Menyimpan hasil eksperimen

# Menampilkan hasil akhir eksperimen
print("\nHasil Akhir Eksperimen:")
for result in results:
    print(f"Kernel Size: {result[0]}, Pooling: {result[1]}, Optimizer: {result[2]}, Epochs: {result[3]}, Accuracy: {result[4]:.2f}%")

Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=5
Evaluating with params: {'epochs': 5, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5, Loss: 1.6422, Val Loss: 1.3193
Epoch 2/5, Loss: 1.2552, Val Loss: 1.0631
Epoch 3/5, Loss: 1.0936, Val Loss: 1.0783
Epoch 4/5, Loss: 0.9924, Val Loss: 0.9088
Epoch 5/5, Loss: 0.9033, Val Loss: 0.8931
Accuracy: 69.14%
Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=50
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 1.6442, Val Loss: 1.3639
Epoch 2/50, Loss: 1.2624, Val Loss: 1.0656
Epoch 3/50, Loss: 1.0961, Val Loss: 0.9621
Epoch 4/50, Loss: 0.9875, Val Loss: 0.9167
Epoch 5/50, Loss: 0.8936, Val Loss: 0.8682
Epoch 6/50, Loss: 0.8316, Val Loss: 0.8271
Epoch 7/50, Loss: 0.7589, Val Loss: 0.8658
Epoch 8/50, Loss: 0.7015, Val Loss: 0.8980
Epoch 9/50, Loss: 0.6610, Val Loss: 0.8847
Epoch 10/50, Loss: 0.6196, Val Loss: 0.8645
Epoch 11/50, Loss: 0.5819, Val Loss: 0.9045
Epoch 12/50, Loss: 0.5471, Val Loss: 0.9413
Epoch 13/50, Loss: 0.3995, Val Loss: 0.9172
Epoch 14/50, Loss: 0.34